In [1]:
import pandas as pd
import numpy as np
# Libraries for text preprocessing
import re
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
#nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument


In [2]:
df_movies = pd.read_csv('./datasets/movies_imdb.csv')

In [54]:
df_movies.head()

,cast,color,directors,genres,movieId,plots,producers,rating,runtimes,synopsis,title,writers,year
0,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,Color::(Technicolor),John Lasseter,Animation|Adventure|Comedy|Family|Fantasy,1,A cowboy doll is profoundly threatened and jea...,Bonnie Arnold|Ed Catmull|Ralph Guggenheim|Stev...,8.3,81.0,A boy called Andy Davis (voice: John Morris) u...,Toy Story,John Lasseter|Pete Docter|Andrew Stanton|Joe R...,1995.0
1,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Color::(Technicolor),Joe Johnston,Adventure|Comedy|Family|Fantasy,2,When two kids find and play a magical board ga...,Robert W. Cort|Ted Field|Larry Franco|Scott Kr...,7.0,104.0,The film begins in 1869 in the town of Brantfo...,Jumanji,Jonathan Hensleigh|Greg Taylor|Jim Strain|Greg...,1995.0
2,Walter Matthau|Jack Lemmon|Sophia Loren|Ann-Ma...,Color::(Technicolor),Howard Deutch,Comedy|Romance,3,John and Max resolve to save their beloved bai...,Richard C. Berman|John Davis|George Folsey Jr....,6.7,101.0,NaN,Grumpier Old Men,Mark Steven Johnson|Mark Steven Johnson,1995.0
3,Whitney Houston|Angela Bassett|Loretta Devine|...,Color,Forest Whitaker,Comedy|Drama|Romance,4,"Based on Terry McMillan's novel, this film fol...",Ronald Bass|Caron K|Terry McMillan|Deborah Sch...,5.9,124.0,NaN,Waiting to Exhale,Terry McMillan|Terry McMillan|Ronald Bass,1995.0
4,Steve Martin|Diane Keaton|Martin Short|Kimberl...,Color::(Technicolor),Charles Shyer,Comedy|Family|Romance,5,George Banks must deal not only with the pregn...,Carol Baum|Bruce A. Block|Julie B. Crane|Jim C...,6.0,106.0,NaN,Father of the Bride Part II,Albert Hackett|Frances Goodrich|Nancy Meyers|C...,1995.0


In [55]:
df_movies.isna().sum()

cast           310
color          128
directors      280
genres          15
movieId          0
plots          366
producers     1055
rating          23
runtimes        74
synopsis     18641
title            2
writers       1328
year             2
dtype: int64

In [3]:
df_movies.dropna(subset=['plots']).isna().sum()

cast           281
color          115
directors      267
genres           8
movieId          0
plots            0
producers      986
rating          12
runtimes        54
synopsis     18312
title            0
writers       1260
year             0
dtype: int64

In [57]:
df_movies.loc[0,'plots']

'A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy\'s room.|A little boy named Andy loves to be in his room, playing with his toys, especially his doll named "Woody". But, what do the toys do when Andy is not with them, they come to life. Woody believes that his life (as a toy) is good. However, he must worry about Andy\'s family moving, and what Woody does not know is about Andy\'s birthday party. Woody does not realize that Andy\'s mother gave him an action figure known as Buzz Lightyear, who does not believe that he is a toy, and quickly becomes Andy\'s new favorite toy. Woody, who is now consumed with jealousy, tries to get rid of Buzz. Then, both Woody and Buzz are now lost. They must find a way to get back to Andy before he moves without them, but they will have to pass through a ruthless toy killer, Sid Phillips.|Woody, a good-hearted cowboy doll who belongs to a young boy named Andy, sees his position as Andy\'s favo

In [4]:
df_movies['plots'] = df_movies['plots'].apply(lambda x: str(x).replace("|"," "))

In [5]:
##Creating a list of stop words and adding custom stopwords
#nltk.download('wordnet')
#nltk.download('stopwords')
stop_words = set(stopwords.words("english"))

In [2]:
docs = []
for i in range(0, df_movies.shape[0]):
    #Remove punctuations
    text = re.sub('[^a-zA-Z]', ' ', df_movies['plots'][i])
    
    #Convert to lowercase
    text = text.lower()
    
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    ##Convert to list from string
    text = text.split()
    
    ##Stemming
    ps=PorterStemmer()
    #Lemmatisation
    lem = WordNetLemmatizer()
    words = [lem.lemmatize(word) for word in text if not word in  
            stop_words] 
    #text = " ".join(text)
    docs.append(TaggedDocument(words=words, tags=[df_movies['title'][i]]))

NameError: name 'df_movies' is not defined

In [7]:
docs[0]

TaggedDocument(words=['cowboy', 'doll', 'profoundly', 'threatened', 'jealous', 'new', 'spaceman', 'figure', 'supplants', 'top', 'toy', 'boy', 'room', 'little', 'boy', 'named', 'andy', 'love', 'room', 'playing', 'toy', 'especially', 'doll', 'named', 'woody', 'toy', 'andy', 'come', 'life', 'woody', 'belief', 'life', 'toy', 'good', 'however', 'must', 'worry', 'andy', 'family', 'moving', 'woody', 'know', 'andy', 'birthday', 'party', 'woody', 'realize', 'andy', 'mother', 'gave', 'action', 'figure', 'known', 'buzz', 'lightyear', 'believe', 'toy', 'quickly', 'becomes', 'andy', 'new', 'favorite', 'toy', 'woody', 'consumed', 'jealousy', 'try', 'get', 'rid', 'buzz', 'woody', 'buzz', 'lost', 'must', 'find', 'way', 'get', 'back', 'andy', 'move', 'without', 'pas', 'ruthless', 'toy', 'killer', 'sid', 'phillips', 'woody', 'good', 'hearted', 'cowboy', 'doll', 'belongs', 'young', 'boy', 'named', 'andy', 'see', 'position', 'andy', 'favorite', 'toy', 'jeopardized', 'parent', 'buy', 'buzz', 'lightyear', '

In [9]:
import multiprocessing
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

cores = multiprocessing.cpu_count()
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(docs)])

100%|██████████| 27278/27278 [00:00<00:00, 2295268.01it/s]


In [10]:
from sklearn import utils

for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(docs)]), total_examples=len(docs), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 27278/27278 [00:00<00:00, 4546843.56it/s]


In [69]:
X = np.array([model_dbow.infer_vector(doc.words, steps=20) for doc in docs])
X.shape

(27278, 300)

In [74]:
from sklearn.metrics.pairwise import cosine_similarity
cos_s = cosine_similarity(X[0].reshape(1,-1), X)

In [75]:
df_sim = pd.DataFrame(cos_s.T, index=df_movies['title'])

In [76]:
df_sim.sort_values(by=0, ascending=False)

,0
title,
Jumanji,1.000000
Torment,0.996873
The Boost,0.996668
Captain January,0.996377
A Night in the Life of Jimmy Reardon,0.996298
Heart and Souls,0.996212
White Chicks,0.996208
The Birth of a Nation,0.996188
Be Kind Rewind,0.996182


In [77]:
df_ratings = pd.read_csv('./datasets/ml-20m/ratings.csv')

In [78]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [80]:
df_ratings_1 = df_ratings[df_ratings['userId'] == 1]
df_ratings_1.head()
df_ratings_1 = df_ratings_1.drop(columns=['userId'])
df_ratings_1.index = df_ratings_1['movieId']
df_ratings_1 = df_ratings_1.drop(columns=['movieId', 'timestamp'])
df_movies_1 = pd.DataFrame(X, index=df_movies['movieId'])
df_movies_1['title'] = df_movies['title'] 
df_movies_1['rating'] = df_ratings_1['rating']
df_movies_1 = df_movies_1[df_movies_1['rating'].notna()]
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_movies_1 , test_size=0.2)
X_train = train.drop(columns=['rating','title'])
y_train = train['rating']
X_test = test.drop(columns=['rating','title'])
y_test = test['rating']

In [81]:
from sklearn.linear_model import LassoCV, RidgeCV, BayesianRidge
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
#lasso = LassoCV(cv=5).fit(X_train, y_train)
#print('LassoCV:',mean_squared_error(y_test, lasso.predict(X_test)))
ridge = RidgeCV(cv=5).fit(X_train, y_train)
print('RidgeCV:',mean_squared_error(y_test, ridge.predict(X_test)))
bayesian = BayesianRidge().fit(X_train, y_train)
print('BayesianRidge:',mean_squared_error(y_test, bayesian.predict(X_test)))
kr = KernelRidge().fit(X_train, y_train)
print('KernelRidge:',mean_squared_error(y_test, kr.predict(X_test)))
ada = AdaBoostRegressor(random_state=0, n_estimators=100).fit(X_train, y_train)
print('AdaBoostRegressor:',mean_squared_error(y_test, ada.predict(X_test)))
gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,max_depth=1, random_state=0, loss='ls').fit(X_train, y_train)
print('GradientBoostingRegressor:',mean_squared_error(y_test, gbr.predict(X_test)))

RidgeCV: 0.6678475377984792
BayesianRidge: 0.5907867747704723
KernelRidge: 1.771956257706761
AdaBoostRegressor: 0.15282671495979966
GradientBoostingRegressor: 0.15684823109915802


In [59]:
movies = pd.DataFrame(ridge.predict(X_test), index=test['title'])

In [60]:
movies.sort_values(by=0, ascending=False)

,0
title,
Othello,3.886033
The Yards,3.824876
"Bon Voyage, Charlie Brown (and Don't Come Back!!)",3.821023
Ghosts of Mississippi,3.790643
The Dark Mirror,3.780490
Strawberry and Chocolate,3.777942
Somewhere in Time,3.776485
C.H.U.D.,3.775172
Un Chien Andalou,3.773966


In [61]:
movies_true = pd.DataFrame(y_test.values, index=test['title'])
movies_true.sort_values(by=0, ascending=False)

,0
title,
Monsoon Wedding,4.0
Blood and Wine,4.0
The Quiet Man,4.0
Top Gun,4.0
Strawberry and Chocolate,4.0
The Evil Dead,4.0
Somewhere in Time,4.0
The Lodger: A Story of the London Fog,4.0
C.H.U.D.,4.0


In [97]:
y_test

movieId
2699    3.0
2694    4.0
1097    5.0
2034    4.0
2916    4.0
1196    5.0
318     5.0
780     3.0
589     4.0
2808    4.0
2797    4.0
1228    3.0
2948    4.0
2448    5.0
2118    5.0
3039    5.0
1258    5.0
1288    2.0
924     5.0
1257    5.0
1266    5.0
2140    4.0
2531    3.0
490     5.0
1810    3.0
1221    5.0
1831    5.0
1779    3.0
1       4.0
904     5.0
2541    1.0
1676    5.0
1214    5.0
2455    4.0
1674    4.0
905     3.0
1089    5.0
1272    4.0
Name: rating, dtype: float64